#### Imports


In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np


#### Carregando os arquivos


In [2]:
df_relatorio_vendas = (
    pd.read_csv('./dataset/relatorio_vendas_etapa_6.csv', sep=';')
)


#### Preparação dos dados


**Iremos inicialmente separar o relatório em duas parte:**

1. 
2. Vendas em dias comuns.<br>
   **De posse destes resultaods, iremos filtra cada dataframe utilizando a coluna IN_IS_SUMMER**<br>
3. Vendas em dias de feriado + verão<br>
4. Vendas em dias comuns + verão<br>

**Próximo passo será separar para cada datframe os 25 campeões de venda**


In [3]:
# Separando as Vendas em dias de feriado.
df_holiday = (
    df_relatorio_vendas[
        df_relatorio_vendas['IN_IS_HOLIDAY'] == 1
    ]
)

# Separando as Vendas em dias de feriado + verão.
df_summer_holiday = (
    df_holiday[
        df_holiday['IN_IS_SUMMER'] == 1
    ]
)

# Separando as Vendas em dias comuns.
df_non_holiday = (
    df_relatorio_vendas[
        df_relatorio_vendas['IN_IS_HOLIDAY'] == 0
    ]
)

# Separando as Vendas em dias de comuns + verão.
df_summer_non_holiday = (
    df_non_holiday[
        df_non_holiday['IN_IS_SUMMER'] == 1
    ]
)


#### Separando os 25 campeões de vendas.


In [7]:
# Mostrando a quantidade vendida por todos os produtos
(
    df_relatorio_vendas
    .groupby(
        [
            'CD_PRODUCT_COLOR'
        ]
    )
    .agg(
        DATA_VOLUME=(
            'DT_ORDER_DATE', 'count'
        )
    )
    .sort_values(
        'DATA_VOLUME'
        , ascending=False
    )
)


,DATA_VOLUME
CD_PRODUCT_COLOR,
3b2db85acef3,6093
98ab6658ac0e,3450
f85b9e0d83ee,2642
e6611bc73de1,2507
d4ce4adf4baa,2493
...,...
525f3de71a2d,1
7ad7ea85fb27,1
835f514c3ad0,1


In [ ]:

# Separando os 25 campeões de vendas
top_25_products = (
    df_relatorio_vendas
    .groupby('CD_PRODUCT_COLOR')
    .agg(
        VL_TOTAL_SOLD=(
            'QT_SOLD', 'sum'
        )
    )
    .sort_values('VL_TOTAL_SOLD', ascending=False)
    .head(25)
    .index
)


# Mostrando a quantidade de vendas dos 25 campeões de venda.
(
    df_relatorio_vendas[
        df_relatorio_vendas['CD_PRODUCT_COLOR']
        .isin(top_25_products)
    ]
    .groupby(
        [
            'CD_PRODUCT_COLOR'
        ]
    )
    .agg(
        DATA_VOLUME=(
            'DT_ORDER_DATE', 'count'
        )
    )
    .sort_values('DATA_VOLUME', ascending=False)
)


#### Filtrando os dataframes por cenário

In [10]:
df_holiday = (
    df_holiday[
        df_holiday['CD_PRODUCT_COLOR']
        .isin(top_25_products)
    ]
)

df_summer_holiday = (
    df_summer_holiday[
        df_summer_holiday['CD_PRODUCT_COLOR']
        .isin(top_25_products)
    ]
)

df_non_holiday = (
    df_non_holiday[
        df_non_holiday['CD_PRODUCT_COLOR']
        .isin(top_25_products)
    ]
)

df_summer_non_holiday = (
    df_summer_non_holiday[
        df_summer_non_holiday['CD_PRODUCT_COLOR']
        .isin(top_25_products)
    ]
)


In [14]:
# Searching for the sold amount for each product
df_sold_amount = (df_relatorio_vendas
                  .groupby('CD_PRODUCT_COLOR', as_index=False)
                  .agg(QT_TOTAL_SOLD_AMOUNT=('QT_SOLD', 'sum'))
                  ).sort_values('QT_TOTAL_SOLD_AMOUNT', ascending=False)


#### Modelos


**`Vendas em feriados`**    

In [21]:
results_values = {
    "CD_PRODUCT_COLOR": [],
    "VL_PRICE_ELASTICITY": [],
    "NR_P_VALUE": [],
}

# Append x_values with y_values per same product name
for product in df_holiday['CD_PRODUCT_COLOR'].unique():
    df = df_holiday[(df_holiday['CD_PRODUCT_COLOR'] == product)].reset_index()

    # Linear Regression Model
    x_value = df['VL_DISCOUNT_PRICE_A2'].values
    y_value = df['QT_SOLD'].values
    X = sm.add_constant(x_value, has_constant='add')

    model = sm.OLS(y_value, X)
    result = model.fit()
    ypred = result.predict(X)

    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)

    if coefficient_pvalue <= 0.05:

        # Price elasticity Formula
        price_elasticity = (slope)*(mean_price/mean_quantity)

        # Append results into dictionary for dataframe
        results_values["CD_PRODUCT_COLOR"].append(product)
        results_values["VL_PRICE_ELASTICITY"].append(price_elasticity)
        results_values["NR_P_VALUE"].append(coefficient_pvalue)


df_OLS_price_elasticity = pd.DataFrame.from_dict(results_values)

df_OLS_price_elasticity = (
    df_OLS_price_elasticity[[
        'CD_PRODUCT_COLOR', 'VL_PRICE_ELASTICITY', 'NR_P_VALUE'
    ]]
)

df_OLS_price_elasticity

,CD_PRODUCT_COLOR,VL_PRICE_ELASTICITY,NR_P_VALUE
0,b678d45011fc,0.579049,0.049762
1,a42e1e23fd85,0.569072,0.010605
2,b4db25ac4d02,0.408017,0.000489
3,f4963f878078,0.343225,0.026263
4,c8b65ec7185d,0.330958,0.032901
5,e0c8654448f1,0.374479,0.027267


**`Vendas em feriados + verão`**    

In [ ]:
results_values = {
    "CD_PRODUCT_COLOR": [],
    "VL_PRICE_ELASTICITY": [],
    "NR_P_VALUE": [],
}

# Append x_values with y_values per same product name
for product in df_summer_holiday['CD_PRODUCT_COLOR'].unique():
    df = df_summer_holiday[(df_summer_holiday['CD_PRODUCT_COLOR'] == product)].reset_index()

    # Linear Regression Model
    x_value = df['VL_DISCOUNT_PRICE_A2'].values
    y_value = df['QT_SOLD'].values
    X = sm.add_constant(x_value, has_constant='add')

    model = sm.OLS(y_value, X)
    result = model.fit()
    ypred = result.predict(X)

    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)

    if coefficient_pvalue <= 0.05:

        # Price elasticity Formula
        price_elasticity = (slope)*(mean_price/mean_quantity)

        # Append results into dictionary for dataframe
        results_values["CD_PRODUCT_COLOR"].append(product)
        results_values["VL_PRICE_ELASTICITY"].append(price_elasticity)
        results_values["NR_P_VALUE"].append(coefficient_pvalue)


df_OLS_price_elasticity = pd.DataFrame.from_dict(results_values)

df_OLS_price_elasticity = (
    df_OLS_price_elasticity[[
        'CD_PRODUCT_COLOR', 'VL_PRICE_ELASTICITY', 'NR_P_VALUE'
    ]]
)

df_OLS_price_elasticity

**`Vendas em dias comuns`**    

In [23]:
results_values = {
    "CD_PRODUCT_COLOR": [],
    "VL_PRICE_ELASTICITY": [],
    "NR_P_VALUE": [],
}

# Append x_values with y_values per same product name
for product in df_non_holiday['CD_PRODUCT_COLOR'].unique():
    df = df_non_holiday[(df_non_holiday['CD_PRODUCT_COLOR']
                         == product)].reset_index()

    # Linear Regression Model
    x_value = df['VL_DISCOUNT_PRICE_A2'].values
    y_value = df['QT_SOLD'].values
    X = sm.add_constant(x_value, has_constant='add')

    model = sm.OLS(y_value, X)
    result = model.fit()
    ypred = result.predict(X)

    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)

    if coefficient_pvalue <= 0.05:

        # Price elasticity Formula
        price_elasticity = (slope)*(mean_price/mean_quantity)

        # Append results into dictionary for dataframe
        results_values["CD_PRODUCT_COLOR"].append(product)
        results_values["VL_PRICE_ELASTICITY"].append(price_elasticity)
        results_values["NR_P_VALUE"].append(coefficient_pvalue)


df_OLS_price_elasticity = pd.DataFrame.from_dict(results_values)

df_OLS_price_elasticity = (
    df_OLS_price_elasticity[[
        'CD_PRODUCT_COLOR', 'VL_PRICE_ELASTICITY', 'NR_P_VALUE'
    ]]
)

df_OLS_price_elasticity

,CD_PRODUCT_COLOR,VL_PRICE_ELASTICITY,VL_MEAN_PRICE,NR_P_VALUE
0,5d7ad4166a0d,0.166559,124.153941,1.852788e-02
1,3b2db85acef3,0.290656,124.415005,2.597728e-08
2,b4db25ac4d02,0.227310,123.331967,9.014788e-04
3,f4963f878078,0.222464,123.688048,1.405501e-03
4,bf36b612091e,0.333656,121.418768,2.576640e-04
5,c2ae5e81d1a0,0.263880,126.934228,8.050548e-04
6,c5d04626700b,0.272431,121.511186,8.687305e-03
7,f9644751d140,0.165176,126.455724,8.764911e-03
8,48f3ad7ee764,0.342487,124.088973,2.285123e-03
9,ffea2321c049,0.175171,124.081879,5.027763e-03


**`Vendas em dias comuns + verão`**    

In [24]:
results_values = {
    "CD_PRODUCT_COLOR": [],
    "VL_PRICE_ELASTICITY": [],
    "NR_P_VALUE": [],
}

# Append x_values with y_values per same product name
for product in df_summer_non_holiday['CD_PRODUCT_COLOR'].unique():
    df = df_summer_non_holiday[(
        df_summer_non_holiday['CD_PRODUCT_COLOR'] == product)].reset_index()

    # Linear Regression Model
    x_value = df['VL_DISCOUNT_PRICE_A2'].values
    y_value = df['QT_SOLD'].values
    X = sm.add_constant(x_value, has_constant='add')

    model = sm.OLS(y_value, X)
    result = model.fit()
    ypred = result.predict(X)

    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)

    if coefficient_pvalue <= 0.05:

        # Price elasticity Formula
        price_elasticity = (slope)*(mean_price/mean_quantity)

        # Append results into dictionary for dataframe
        results_values["CD_PRODUCT_COLOR"].append(product)
        results_values["VL_PRICE_ELASTICITY"].append(price_elasticity)
        results_values["NR_P_VALUE"].append(coefficient_pvalue)


df_OLS_price_elasticity = pd.DataFrame.from_dict(results_values)

df_OLS_price_elasticity = (
    df_OLS_price_elasticity[[
        'CD_PRODUCT_COLOR', 'VL_PRICE_ELASTICITY', 'NR_P_VALUE'
    ]]
)

df_OLS_price_elasticity

,CD_PRODUCT_COLOR,VL_PRICE_ELASTICITY,VL_MEAN_PRICE,NR_P_VALUE
0,3b2db85acef3,0.252026,122.709217,2.411036e-07
1,bf36b612091e,0.362517,118.302766,8.112073e-05
2,c2ae5e81d1a0,0.319785,126.259310,8.964732e-03
3,c5d04626700b,0.379994,123.475714,4.103686e-02
4,b4db25ac4d02,0.327107,123.235103,9.401618e-03
5,18c09fe5731e,0.200549,122.811398,4.959898e-02
6,a42e1e23fd85,0.238312,124.299355,1.621915e-02
7,2780e2d2165f,0.276209,126.787513,2.510767e-04
8,f1be40951e0f,0.301401,123.983101,1.497059e-02
9,e0c8654448f1,0.467125,120.466770,1.300305e-04
